# Player evaluation

Here we compare players based on their adjusted points. Also, we calculate some useful metrics using the adjusted points to further evaluate player performance in FPL.

In [1]:
# recent gameweek
gameweek = 34

In [2]:
# import basic libraries
import pandas as pd
import numpy as np

pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/data_week' + str(gameweek) + '.csv'
df = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch team data
filepath = '../data/fbref/team_stats_week' + str(gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, encoding='latin-1')

# fetch player stats data
filepath = '../data/fbref/player_stats_week' + str(gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)#, encoding='latin-1')

## Value and value points

Here we calculate two interesting metrics: 'value' and 'value points'. 

Value is simply adjusted points per game divided by the cost of the player. Essentially, this measures how many 'points per pound(/euro/whatever)' has the player gained on an average game week. 

The value points is calculated as the geometric mean of a players adjusted points per game and value. Essentially, in FPL we want to gain the maximum possible amount of points. However, we are restrained by a limited budget, so that in practice we cannot just pick all the best players. Then, we would like to have a squad of players that give the most points given our budget. Thus, we would like to pick players that have high value ('bang for buck'). However, often the players with highest value are very cheap players who somewhat overperform relative to their price. Picking a squad full of these players might leave a part of our budget unused. Calculating the geometric mean of adjusted points and value allows us to weigh both aspects equally, i.e. we want players that both gain a lot of points but also have good value. For example, this metric values equally two players, where one has twice the points per game of the other who in turn has twice the value of the other.

In [4]:
# value = expected points / cost
df['value'] = df['adjusted points per game'] / (df['now_cost'] / 10.0)
# geometric mean of 'adjusted points per game' and 'value'
df['valuePoints metric'] = np.sqrt(df['adjusted points per game'] * df['value'])

In [5]:
# assign proper team names for each player
team_names = np.sort(teamStats['Squad'])
df['team_name'] = team_names[df['team']-1]

In [6]:
# save data
filepath = '../data/data_week' + str(gameweek) + str('.csv')
df.to_csv(filepath)

In [7]:
goalkeepers = df['element_type'] == 1
defenders = df['element_type'] == 2
midfielders = df['element_type'] == 3
forwards = df['element_type'] == 4

minGames = df['games played'] >= 3

## Player evaluation

Below we compile lists for each position sorting players based on a given metric.

In [13]:
#metric = 'valuePoints metric'
metric = 'adjusted points per game'
#metric = 'form 10'

numberToShow = 40

print('GOALKEEPERS')
display(df[goalkeepers & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('DEFENDERS')
display(df[defenders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('MIDFIELDERS')
display(df[midfielders & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))
print('FORWARDS')
display(df[forwards & minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

GOALKEEPERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
340,Lloris,Tottenham,53,17.021277,80,76.350981,4.7,4.485620,4.300277,4.660794,1.948429
411,Patrício,Wolves,53,34.250000,137,148.536460,4.0,4.336831,3.831366,4.180220,1.883799
168,Schmeichel,Leicester City,54,33.636364,148,141.905298,4.4,4.218806,5.294507,5.478951,1.815485
471,Henderson,Sheffield Utd,52,31.956522,147,133.949232,4.6,4.191609,4.441116,4.356970,1.838142
93,Pope,Burnley,50,34.130435,157,140.557712,4.6,4.118251,4.632083,4.523549,1.841738
235,de Gea,Manchester Utd,53,34.444444,124,133.598680,3.6,3.878671,4.138105,4.280180,1.684787
189,Alisson,Liverpool,62,25.217391,116,97.379093,4.6,3.861585,3.740290,3.704136,1.550850
131,Guaita,Crystal Palace,49,31.219512,128,120.089027,4.1,3.846602,2.720265,3.836324,1.737717
14,Leno,Arsenal,48,30.000000,114,114.649581,3.8,3.821653,3.399736,4.404569,1.744338


DEFENDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
182,Alexander-Arnold,Liverpool,78,34.210526,195,178.498853,5.7,5.217659,3.322212,3.987117,1.868222
103,Alonso,Chelsea,61,13.965517,81,66.315771,5.8,4.748537,3.092702,4.605887,1.922626
401,Doherty,Wolves,64,32.222222,145,151.859650,4.5,4.712886,4.619437,5.362978,1.862932
181,Robertson,Liverpool,70,31.836735,156,146.344042,4.9,4.596704,2.632830,3.062674,1.737391
105,Azpilicueta,Chelsea,60,31.944444,115,144.368272,3.6,4.519355,5.006606,4.873782,1.845019
297,Lundstram,Sheffield Utd,46,29.777778,134,129.435839,4.5,4.346726,0.941619,3.439111,2.026671
183,van Dijk,Liverpool,65,33.673469,165,141.042117,4.9,4.188523,3.277431,3.566011,1.642874
122,Wan-Bissaka,Manchester Utd,55,30.810811,114,124.151907,3.7,4.029492,4.353264,4.575269,1.718181
159,Pereira,Leicester City,62,27.954545,123,112.583824,4.4,4.027389,NaN,4.111600,1.617438


MIDFIELDERS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
618,Fernandes,Manchester Utd,90,10.000000,94,68.861463,9.4,6.886146,8.382868,7.690343,2.295382
191,Salah,Liverpool,125,30.135135,223,205.477044,7.4,6.818521,7.844354,6.676319,1.928569
215,De Bruyne,Manchester City,108,31.857143,223,202.645592,7.0,6.361072,7.207756,7.017348,1.935613
192,Mané,Liverpool,123,30.769231,200,178.233778,6.5,5.792598,4.010083,4.456057,1.651661
214,Sterling,Manchester City,117,29.074074,157,164.006566,5.4,5.640990,5.488553,5.747270,1.649159
342,Son,Tottenham,97,25.925926,140,126.866841,5.4,4.893435,3.799729,4.208254,1.571187
431,Pulisic,Chelsea,73,20.980392,107,102.439220,5.1,4.882617,5.961123,6.068176,1.807138
239,Martial,Manchester Utd,83,28.000000,168,136.223871,6.0,4.865138,5.429691,5.421779,1.688714
219,David Silva,Manchester City,73,23.921569,122,115.081535,5.1,4.810785,7.684165,6.580889,1.780552


FORWARDS


,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
233,Rashford,Manchester Utd,90,27.192982,155,151.573109,5.7,5.573979,5.122410,5.173005,1.857993
166,Vardy,Leicester City,98,30.952381,195,171.411236,6.3,5.537901,7.311759,6.009492,1.769018
11,Aubameyang,Arsenal,109,31.724138,184,160.427491,5.8,5.056954,6.258567,5.692534,1.531707
409,Jiménez,Wolves,82,34.000000,170,166.371236,5.0,4.893272,3.331759,3.884083,1.708805
210,Agüero,Manchester City,117,24.000000,132,117.071236,5.5,4.877968,6.968516,4.324939,1.426087
187,Firmino,Liverpool,94,33.902439,139,157.814981,4.1,4.654974,3.037900,3.283347,1.518285
313,Ings,Southampton,74,34.038462,177,157.603745,5.2,4.630167,6.065026,4.997566,1.702083
460,Abraham,Chelsea,74,30.000000,144,137.311236,4.8,4.577041,2.836215,3.047518,1.682554
278,Pukki,Norwich City,64,32.682927,134,141.645618,4.1,4.333933,1.749213,2.154951,1.713137


Below we compile a list sorting players based on a given metric (irrespective of position).

In [9]:
metric = 'adjusted points per game'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
618,Fernandes,Manchester Utd,90,10.000000,94,68.861463,9.4,6.886146,8.382868,7.690343,2.295382
191,Salah,Liverpool,125,30.135135,223,205.477044,7.4,6.818521,7.844354,6.676319,1.928569
215,De Bruyne,Manchester City,108,31.857143,223,202.645592,7.0,6.361072,7.207756,7.017348,1.935613
192,Mané,Liverpool,123,30.769231,200,178.233778,6.5,5.792598,4.010083,4.456057,1.651661
214,Sterling,Manchester City,117,29.074074,157,164.006566,5.4,5.640990,5.488553,5.747270,1.649159
233,Rashford,Manchester Utd,90,27.192982,155,151.573109,5.7,5.573979,5.122410,5.173005,1.857993
166,Vardy,Leicester City,98,30.952381,195,171.411236,6.3,5.537901,7.311759,6.009492,1.769018
182,Alexander-Arnold,Liverpool,78,34.210526,195,178.498853,5.7,5.217659,3.322212,3.987117,1.868222
11,Aubameyang,Arsenal,109,31.724138,184,160.427491,5.8,5.056954,6.258567,5.692534,1.531707


In [10]:
metric = 'form 10'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
618,Fernandes,Manchester Utd,90,10.000000,94,68.861463,9.4,6.886146,8.382868,7.690343,2.295382
113,Willian,Chelsea,73,33.125000,159,149.438463,4.8,4.511350,8.934136,7.144007,1.669725
215,De Bruyne,Manchester City,108,31.857143,223,202.645592,7.0,6.361072,7.207756,7.017348,1.935613
191,Salah,Liverpool,125,30.135135,223,205.477044,7.4,6.818521,7.844354,6.676319,1.928569
219,David Silva,Manchester City,73,23.921569,122,115.081535,5.1,4.810785,7.684165,6.580889,1.780552
431,Pulisic,Chelsea,73,20.980392,107,102.439220,5.1,4.882617,5.961123,6.068176,1.807138
166,Vardy,Leicester City,98,30.952381,195,171.411236,6.3,5.537901,7.311759,6.009492,1.769018
389,Antonio,West Ham,69,19.714286,69,86.943411,3.5,4.410173,6.501734,5.940823,1.678924
214,Sterling,Manchester City,117,29.074074,157,164.006566,5.4,5.640990,5.488553,5.747270,1.649159


In [11]:
metric = 'valuePoints metric'

display(df[minGames].sort_values(by=[metric], ascending=False)[['web_name','team_name','now_cost',\
          'games played','total_points','adjusted points','points_per_game','adjusted points per game','form 5',\
          'form 10','valuePoints metric']].head(numberToShow))

,web_name,team_name,now_cost,games played,total_points,adjusted points,points_per_game,adjusted points per game,form 5,form 10,valuePoints metric
id,,,,,,,,,,,
618,Fernandes,Manchester Utd,90,10.000000,94,68.861463,9.4,6.886146,8.382868,7.690343,2.295382
297,Lundstram,Sheffield Utd,46,29.777778,134,129.435839,4.5,4.346726,0.941619,3.439111,2.026671
340,Lloris,Tottenham,53,17.021277,80,76.350981,4.7,4.485620,4.300277,4.660794,1.948429
215,De Bruyne,Manchester City,108,31.857143,223,202.645592,7.0,6.361072,7.207756,7.017348,1.935613
191,Salah,Liverpool,125,30.135135,223,205.477044,7.4,6.818521,7.844354,6.676319,1.928569
103,Alonso,Chelsea,61,13.965517,81,66.315771,5.8,4.748537,3.092702,4.605887,1.922626
411,Patrício,Wolves,53,34.250000,137,148.536460,4.0,4.336831,3.831366,4.180220,1.883799
341,Gazzaniga,Tottenham,41,18.125000,58,68.611240,3.2,3.785448,NaN,1.653034,1.869499
182,Alexander-Arnold,Liverpool,78,34.210526,195,178.498853,5.7,5.217659,3.322212,3.987117,1.868222
